# Notas de Gonche

In [26]:
import pandas as pd
import geopandas as gpd
import numpy as np



import simulator.constants as con

from sklearn.metrics.pairwise import haversine_distances
from math import radians


In [34]:
world_pop_density_file = "/home/minigonche/Dropbox/Projects/synthetic_mobility_data_generator/data/costa_rica-panama/pd_2020_1km_ASCII_XYZ.csv"
populated_places_folder = "/home/minigonche/Dropbox/Projects/synthetic_mobility_data_generator/data/costa_rica-panama/hotosm_populated_places_points_shp"
road_lines_folder = "/home/minigonche/Dropbox/Projects/synthetic_mobility_data_generator/data/costa_rica-panama/hotosm_roads_lines_shp"

In [18]:
# Density
world_pop_density = pd.read_csv(world_pop_density_file)
world_pop_density = gpd.GeoDataFrame(world_pop_density, geometry=gpd.points_from_xy(world_pop_density.X, world_pop_density.Y), crs=con.USUAL_PROJECTION)

# Populated Places
populated_places = gpd.read_file(populated_places_folder)

# Uses Haversine Distance
# (A lot faster than geopandas.distance)
world_pop_density["lon_rad"] = world_pop_density.geometry.x.apply(lambda v : radians(v))
world_pop_density["lat_rad"] = world_pop_density.geometry.y.apply(lambda v : radians(v))

populated_places["lon_rad"] = populated_places.geometry.x.apply(lambda v : radians(v))
populated_places["lat_rad"] = populated_places.geometry.y.apply(lambda v : radians(v))

# Extracts closest city
closest_city = world_pop_density.apply(lambda row : np.argmin(haversine_distances(populated_places[["lat_rad", "lon_rad"]], [[row["lat_rad"], row["lon_rad"]]])[:,0]), axis = 1)

# Groups, sums and assigns
world_pop_density["city"] = closest_city
population = world_pop_density[["Z","city"]].groupby("city").sum()


# Forms the final Frame
populated_places["id"] = populated_places.apply(lambda row : f"{row['name']} - {row.name}", axis = 1)
populated_places["population"] = population
populated_places["lat"] = populated_places.geometry.y
populated_places["lon"] = populated_places.geometry.x



In [32]:
nodes = populated_places[["id","population","lat","lon","geometry"]]
nodes


,id,population,lat,lon,geometry
0,Liberia - 0,58725.820929,10.633340,-85.436272,POINT (-85.43627 10.63334)
1,Tilarán - 1,10366.156730,10.470980,-84.967260,POINT (-84.96726 10.47098)
2,La Fortuna - 2,1934.716064,10.471684,-84.644436,POINT (-84.64444 10.47168)
3,Limón - 3,19242.702255,9.993560,-83.030314,POINT (-83.03031 9.99356)
4,Sixaola - 4,1591.918549,9.505422,-82.614567,POINT (-82.61457 9.50542)
...,...,...,...,...,...
6550,Bajo La Concepción - 6550,275.206904,8.266004,-81.312094,POINT (-81.31209 8.26600)
6551,Alto de La Palma - 6551,314.413401,8.246557,-81.368233,POINT (-81.36823 8.24656)
6552,Agua Viva - 6552,300.897919,8.217208,-81.294220,POINT (-81.29422 8.21721)
6553,El Bongo - 6553,461.659553,7.920442,-80.462811,POINT (-80.46281 7.92044)


In [35]:

roads = gpd.read_file(road_lines_folder)
roads


,osm_id,highway,width,name,lanes,smoothness,surface,layer,source,bridge,oneway,geometry
0,1.256998e+07,service,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-84.14531 9.95885, -84.14520 9.958..."
1,1.256998e+07,service,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-84.14514 9.95930, -84.14293 9.959..."
2,1.257004e+07,secondary,NaN,Avenida 27,1,NaN,asphalt,NaN,NaN,NaN,no,"LINESTRING (-84.09124 9.94787, -84.09146 9.947..."
3,1.257007e+07,secondary,NaN,Calle 11,2,NaN,asphalt,NaN,NaN,NaN,yes,"LINESTRING (-84.07361 9.91168, -84.07382 9.911..."
4,1.257614e+07,trunk_link,NaN,Entrada Vía 104,NaN,NaN,asphalt,NaN,NaN,NaN,yes,"LINESTRING (-84.11712 9.94019, -84.11713 9.939..."
...,...,...,...,...,...,...,...,...,...,...,...,...
231533,1.133906e+09,residential,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MULTILINESTRING ((-82.60341 9.48720, -82.60237..."
231534,1.133906e+09,residential,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MULTILINESTRING ((-82.60611 9.48635, -82.60563..."
231535,1.133906e+09,residential,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MULTILINESTRING ((-82.60611 9.48635, -82.60563..."
231536,1.133906e+09,residential,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MULTILINESTRING ((-82.60585 9.48664, -82.60541..."


In [38]:
nodes

,id,population,lat,lon,geometry
0,Liberia - 0,58725.820929,10.633340,-85.436272,POINT (-85.43627 10.63334)
1,Tilarán - 1,10366.156730,10.470980,-84.967260,POINT (-84.96726 10.47098)
2,La Fortuna - 2,1934.716064,10.471684,-84.644436,POINT (-84.64444 10.47168)
3,Limón - 3,19242.702255,9.993560,-83.030314,POINT (-83.03031 9.99356)
4,Sixaola - 4,1591.918549,9.505422,-82.614567,POINT (-82.61457 9.50542)
...,...,...,...,...,...
6550,Bajo La Concepción - 6550,275.206904,8.266004,-81.312094,POINT (-81.31209 8.26600)
6551,Alto de La Palma - 6551,314.413401,8.246557,-81.368233,POINT (-81.36823 8.24656)
6552,Agua Viva - 6552,300.897919,8.217208,-81.294220,POINT (-81.29422 8.21721)
6553,El Bongo - 6553,461.659553,7.920442,-80.462811,POINT (-80.46281 7.92044)


In [45]:

nodes.sjoin(roads)

,id,population,lat,lon,geometry,index_right,osm_id,highway,width,name,lanes,smoothness,surface,layer,source,bridge,oneway
3450,Agua Blanca - 3450,233.994013,8.733985,-82.403476,POINT (-82.40348 8.73399),134282,116517947.0,residential,NaN,NaN,NaN,NaN,asphalt,NaN,Bing,NaN,NaN
3450,Agua Blanca - 3450,233.994013,8.733985,-82.403476,POINT (-82.40348 8.73399),134037,92299161.0,residential,NaN,NaN,NaN,NaN,NaN,NaN,Bing,NaN,NaN
3502,El Toro - 3502,418.573555,7.753350,-80.869516,POINT (-80.86952 7.75335),143168,206773221.0,tertiary,NaN,NaN,NaN,NaN,paved,NaN,NaN,NaN,NaN
3502,El Toro - 3502,418.573555,7.753350,-80.869516,POINT (-80.86952 7.75335),157435,525394048.0,unclassified,NaN,NaN,NaN,bad,unpaved,NaN,NaN,NaN,NaN
3502,El Toro - 3502,418.573555,7.753350,-80.869516,POINT (-80.86952 7.75335),143104,195609917.0,unclassified,NaN,NaN,NaN,NaN,ground,NaN,NaN,NaN,no
3807,La Estrella - 3807,355.655067,8.728444,-82.386340,POINT (-82.38634 8.72844),182031,643909343.0,unclassified,NaN,NaN,NaN,NaN,asphalt,NaN,Bing,NaN,NaN
4272,nan - 4272,3715.131548,8.921979,-79.800929,POINT (-79.80093 8.92198),149955,366575859.0,residential,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,yes
4437,El Macano (p) - 4437,289.112314,8.610965,-82.388842,POINT (-82.38884 8.61097),185494,679889151.0,track,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4559,Camiseta - 4559,700.610203,8.796746,-82.489125,POINT (-82.48912 8.79675),182348,654037322.0,unclassified,NaN,Cima del Volcán Barú (PNVB),NaN,NaN,dirt,NaN,NaN,NaN,NaN
4559,Camiseta - 4559,700.610203,8.796746,-82.489125,POINT (-82.48912 8.79675),182381,655070028.0,track,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
